In [ ]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision
from torchvision import models
import torch.optim as optim
import pandas as pd
import numpy as np
import cv2
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt
import re

In [ ]:
def initialize_lfw():
    # output_file_path = "../../data/attribute classifier/lfw/"
    global output_file_path, output_low_level_path, df_attributes
    output_file_path = "/content/drive/MyDrive/project-sepnu/data/neural network/"
    output_low_level_path = '/content/drive/MyDrive/project-sepnu/data/low level/lfw/'
    # output_low_level_path = "../../data/low level/lfw/"
    map_attributes_features_path ="/content/drive/MyDrive/project-sepnu/data/attribute classifier/lfw/map_attributes_features.txt"
    attributes_path = "/content/drive/MyDrive/LFW/metadata/modified_attributes.txt"
    if not os.path.exists(map_attributes_features_path):
        file_attributes = open(attributes_path, "r"); file_map_attributes_features = open(map_attributes_features_path, "w")
        for attribute in file_attributes.readline().replace("\n", "").split(",")[2:]: file_map_attributes_features.write(attribute + ":" + input("Enter space separated features for the attribute \"%s\": " % (attribute, )) + "\n")
        file_attributes.close(); file_map_attributes_features.close()
    file_map_attributes_features = open(map_attributes_features_path, "r")
    global map_attributes_features 
    map_attributes_features = {}
    for attribute, features in [line.split(":") for line in file_map_attributes_features.readlines()]:
        map_attributes_features[attribute] = features.split()
    file_map_attributes_features.close()
    df_attributes = pd.read_csv(attributes_path)
    file_images_under_error = open(output_low_level_path + "images_under_error.txt", "r")
    while True:
        line = file_images_under_error.readline().replace("\n", "")
        if line == "": break
        line = line.split()[0]
        matched_pattern = re.match(r"^([\w-]+?)_(\d+).jpg$", line[line.rindex("/")+1:])
        name, image_num = matched_pattern.groups()
        name = name.replace("_", " ")
        image_num = int(image_num)
        initial_shape = df_attributes.shape
        df_attributes.drop(df_attributes[(df_attributes['person'] == name) & (df_attributes['imagenum'] == image_num)].index, inplace = True)
        final_shape = df_attributes.shape
        assert (final_shape[0] + 1, final_shape[1]) == initial_shape
    file_images_under_error.close()
    assert df_attributes.shape[0] == np.load(output_low_level_path + "rgb_chin.npy").shape[0]
    

    

In [ ]:
class lfwData(Dataset):
    def __init__(self,data_list,attribute,transform=None,train=True):
        super().__init__()
        global map_attributes_features
        self.data_list = data_list
        # self.data_dir = data_dir
        self.transform = transform
        self.train = train
        # self.df_attr = df_attributes
        self.attribute = attribute
        self.features = map_attributes_features[attribute]
        self.map_attr = df_attributes.columns.tolist().index(attribute)

    
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self,item):
        global output_file_path, output_low_level_path, df_attributes
        # print(item)
        img_idx = item
        p1feature = []
        for feature in self.features:
            feature_array = np.load(output_low_level_path + feature + ".npy")
            p1feature.extend(feature_array[img_idx]/feature_array[img_idx].sum())
            # p1feature = np.hstack((p1feature,feature_array[img_idx]))
        
        label = df_attributes.iloc[img_idx,self.map_attr]
        label = max(label,0)
        p1feature = np.array(p1feature)
        if self.transform is not None:
            p1feature = self.transform(p1feature)
        if self.train:
          return {
              'feat' : p1feature,
              'label' : torch.tensor(label)
          }
        else:
          return {
              'feat':p1feature
          }
        

In [ ]:
def get_datalist(df):
    # df = df_attributes
    lfwnames = df['person'].tolist()
    lfwnames_nums = df['imagenum'].tolist()
    lfwnames_act = []
    for i in range(len(lfwnames)):
        num = '{0:04}'.format(lfwnames_nums[i])
        lfwnames_act.append(lfwnames[i].replace(" ","_") + "_" + num + ".jpg")
    assert len(lfwnames_act) == df.shape[0]
    return lfwnames_act


In [ ]:
initialize_lfw()
# print(df_attributes.head())
data_list = get_datalist(df_attributes)

In [ ]:
transforms_train = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.3166, 0.3947, 0.4725), (0.1755, 0.1720, 0.1657)),
])
# transform = transforms.ToTensor()
train_data = lfwData(data_list,"Asian")
# len(train_data)
tem = train_data.__getitem__(3)
tem["feat"].shape
# print(map_attributes_features.keys())

(400,)

In [ ]:
batch = 128
test_size = 0.3
num = train_data.__len__()
indices = list(range(num))
np.random.shuffle(indices)
split = int(np.floor(test_size*num))
train_idx,test_idx = indices[split:], indices[:split]

#Create Samplers
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)
train_loader = DataLoader(train_data, batch_size = batch, sampler = train_sampler)
test_loader = DataLoader(train_data, batch_size = batch, sampler = test_sampler)

In [ ]:
class BasicNet(nn.Module):
	def __init__(self,in_feats):
		super().__init__()
		self.features = nn.Sequential(
			nn.Linear(in_feats, 2048),
			nn.ReLU(),
			nn.Linear(2048, 4096),
			nn.Dropout(p=0.5),
			nn.ReLU(),
            # nn.BatchNorm1d(2048),
            nn.Linear(4096, 8192),
			nn.ReLU(),
            nn.Linear(8192, 4096),
			nn.ReLU(),
            nn.Linear(4096, 2048),
			nn.ReLU(),
			nn.Linear(2048, 1024),
            nn.ReLU(),
			nn.Linear(1024, 512),
            nn.ReLU(),
			nn.Linear(512, 64),
            nn.ReLU(),
			nn.Linear(64, 2),
		)

	def forward(self, x):
		x = self.features(x)
		return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:


print(device)

cuda:0


In [ ]:
model = BasicNet(400).to(device)
error = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters())
print(model)

BasicNet(
  (features): Sequential(
    (0): Linear(in_features=400, out_features=2048, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2048, out_features=4096, bias=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): ReLU()
    (5): Linear(in_features=4096, out_features=8192, bias=True)
    (6): ReLU()
    (7): Linear(in_features=8192, out_features=4096, bias=True)
    (8): ReLU()
    (9): Linear(in_features=4096, out_features=2048, bias=True)
    (10): ReLU()
    (11): Linear(in_features=2048, out_features=1024, bias=True)
    (12): ReLU()
    (13): Linear(in_features=1024, out_features=512, bias=True)
    (14): ReLU()
    (15): Linear(in_features=512, out_features=64, bias=True)
    (16): ReLU()
    (17): Linear(in_features=64, out_features=2, bias=True)
  )
)


In [ ]:
rand_tensor = 8*torch.rand((64,400))
temp = rand_tensor.to(device)
#print tensor
# print(rand_tensor)
# temp = temp.view(200,64)
print(temp.shape)
y = model(temp)
# print(rand_tensor)

torch.Size([64, 400])


In [ ]:
n_epochs = 5
valid_loss_min = np.Inf

train_losses = []
valid_losses = []

for epoch in range(n_epochs):
    train_loss = 0.0
    valid_loss = 0.0
    
    model.train()
    for feats in train_loader:
        data = feats['feat'].squeeze(0).to(device)
        target = feats['label'].to(device)
        optimizer.zero_grad()
        output = model(data.float())
        loss = error(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    
     
    train_loss /= len(train_loader.sampler)
    # valid_loss /= len(valid_loader.sampler)
    
    train_losses.append(train_loss)
    # valid_losses.append(valid_loss)
    
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch, train_loss))
    
    # if valid_loss <= valid_loss_min:
        # print("Validation Loss decreased {:0.6f} -> {:0.6f}".format(valid_loss_min,valid_loss))
        # valid_loss_min = valid_loss/
        # torch.save(model.state_dict, 'best_model_so_far.pth')

Epoch: 0 	Training Loss: 0.294370
Epoch: 1 	Training Loss: 0.272487
Epoch: 2 	Training Loss: 0.276515
Epoch: 3 	Training Loss: 0.260960
Epoch: 4 	Training Loss: 0.247850


In [ ]:
# df_attributes.loc[6043,"Male"]
torch.save(model.state_dict, '/content/drive/MyDrive/project-sepnu/data/neural network/asian_lfw.pth')
# df_attributes.head()

In [ ]:
# model.load_state_dict(torch.load('/content/drive/MyDrive/project-sepnu/data/neural network/Male_lfw.pth'))
model.eval()
correct = 0
total = 0
pred_list = []
correct_list = []
with torch.no_grad():
    for images in test_loader:
        data = images['feat'].squeeze(0).to(device)
        target = images['label'].to(device)
        outputs = model(data.float())
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        pr = predicted.detach().cpu().numpy()
        for i in pr:
          pred_list.append(i)
        tg = target.detach().cpu().numpy()
        for i in tg:
          correct_list.append(i)
        correct += (predicted == target).sum().item()


print('Accuracy of the network on the 10000 test images: %f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 91.854508 %
